# COIN Project

Authors:

    - Mona
    - Roman
    - Nick
    - Mateo

# 1. Setup

### Packages

In [118]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
#import dotenv
import os
import yaml
import datetime
from pprint import pprint
#from datetime import datetime

### Environment Variables

In [119]:
#dotenv.load_dotenv()

In [120]:
with open("../twitter.yml", "r") as f:
    config = yaml.safe_load(f)

In [121]:
config

{'accounts': ['Fridays4future',
  'FridayForFuture',
  'GretaThunberg',
  'Luisamneubauer'],
 'hashtags': {'green': ['EndCoal',
   'EndFossilFuels',
   'PeopleNotProfit',
   'NoMoreEmptyPromises',
   'UprootTheSystem',
   'FridaysForFuture',
   'ClimateAction',
   'ClimateJustice',
   'ClimateEmergency',
   'ClimateStrike',
   'SaveThePlanet'],
  'brown': ['climatescam',
   'climatechangehoax',
   'fakeclimate',
   'climatehoax',
   'globalwarmingisahoax'],
  'neutral': ['ClimateCrisis', 'ClimateChange', 'Climate', 'GlobalWarming']}}

### Database Connection

In [122]:
'''username = os.environ["DB_USERNAME"]
password = os.environ["DB_PASSWORD"]
host = os.environ["DB_HOST"]
port = os.environ["DB_PORT"]
name = os.environ["DB_NAME"]'''

'username = os.environ["DB_USERNAME"]\npassword = os.environ["DB_PASSWORD"]\nhost = os.environ["DB_HOST"]\nport = os.environ["DB_PORT"]\nname = os.environ["DB_NAME"]'

In [123]:
#engine = sa.create_engine("mssql+pymssql://{}:{}@{}/{}".format(username, password, host, name))

### Tweepy API Connection

In [124]:
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAGjtbgEAAAAAyx9oZIai1hXZ9OyDhUUFMZQivpc%3DUunewUXR9hw3nyKQhjqdmfg7zSAoa1nPv6WKLLSPB7OwKwYBP3", wait_on_rate_limit=True)

In [125]:
auth = tweepy.OAuth2BearerHandler(bearer_token="AAAAAAAAAAAAAAAAAAAAAGjtbgEAAAAAyx9oZIai1hXZ9OyDhUUFMZQivpc%3DUunewUXR9hw3nyKQhjqdmfg7zSAoa1nPv6WKLLSPB7OwKwYBP3")
api = tweepy.API(auth, wait_on_rate_limit=True)

# 2. Load Data

### 2.1 Tweets by User

In [126]:
#specify user_ids to look up
users = {user: client.get_user(username=user).data.id for user in config["accounts"]}
users


{'Fridays4future': 1053768884732547072,
 'FridayForFuture': 1072187272815149057,
 'GretaThunberg': 1006419421244678144,
 'Luisamneubauer': 1068337838}

In [127]:
#returns tweets plus metadata for specific user_ids
def get_user_tweets(user_id, start_time='2017-01-01', end_time: str = None, fmt="%Y-%m-%d"):
    expansions = ["author_id"]
    tweet_fields = ["created_at", "geo", "public_metrics", "text", "context_annotations", "entities"]
    start_time = datetime.datetime.strptime(start_time, fmt)
    if end_time:
        end_time = datetime.datetime.strptime(end_time, fmt)
    else: 
        datetime.datetime.now()
    response = tweepy.Paginator(
        client.get_users_tweets,
        id=user_id,
        start_time=start_time,
        end_time=end_time,
        expansions=expansions,
        tweet_fields=tweet_fields
    ).flatten()
    response = list(response)
    tweets = [
        {
            "id": r.id,
            "author_id": r.author_id,
            "created_at": r.created_at,
            "geo": r.geo,
            "retweet_count": r.public_metrics["retweet_count"],
            "reply_count": r.public_metrics["reply_count"],
            "like_count": r.public_metrics["like_count"],
            "qoute_count": r.public_metrics["quote_count"],
            "text": r.text,
            "entities": str(r.entities),
            "context_annotations": str(r.context_annotations)
        } for r in response
    ]
    
    tweets = pd.DataFrame(tweets)
    return tweets

In [128]:
#creates a dataframe with tweets from several users
tweets = pd.DataFrame()

for user in users.values():
    tweets = tweets.append(get_user_tweets(user))

print(tweets.shape)


(12953, 11)


In [129]:
#save dataframe to csv
date = datetime.datetime.now().strftime("%Y_%m_%d-%I:%M:%S_%p")
tweets.to_csv("tweets_by_accounts_{}.csv".format(date), index=False, header=False, mode="a")


### 2.2 Tweets by Hashtags

In [200]:
#specify queries

query = 'from:FridayForFuture'
expansions = ["author_id"]
tweet_fields = ["created_at", "geo", "public_metrics", "text", "context_annotations", "entities"]
max_results = 100
media_fields = ["duration_ms", "height", "media_key", "preview_image_url", "type", "url", "width", "public_metrics", "non_public_metrics", "organic_metrics", "promoted_metrics", "alt_text"]
start_time = '2020-01-01T00:00:00Z' 

In [201]:
def search_all_tweets(query):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                            )
    return response

In [202]:
def search_all_tweets_with_next_token(query, next_token):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,next_token=next_token
                            )
    return response

In [203]:
def fetch_all_data(query, iteration_count, next_token):

    """ Fetches iteration_count * 100 Tweets from Twitter API using the set query params, including the initial next_token
    
    Parameters:
    ------------
    * iteration_count     : Integer indicating the max number of requests 
    * next_token          : String containing the next token for pagination
    
    """
    
    # Initializing the return parameters
    tweets = []
    media = []
    
    for i in range(0, iteration_count):

        if(len(next_token) == 0):
            response = search_all_tweets(query)
        else:
            response = search_all_tweets_with_next_token(query, next_token)

        tweets.append(response.data)
        if "media" in response.includes.keys():
            media.append(response.includes['media'])
        
        try:
            next_token = response.meta['next_token']
        except:
            print('No more Tweets to fetch')
            break


    # Logging the last next_token for pagination into console --> needed for later requesting of the next tweets
    if('next_token' in response.meta):
        print(response.meta['next_token'])
        
    return tweets, media

In [204]:
def unpack_dict(unpack_dict):
    unpacked = [v for k, v in unpack_dict.items()]
    return unpacked
def unpack_attachment(attachment_dict):
    return "|".join(attachment_dict["media_keys"]) if "media_keys" in attachment_dict.keys() else np.nan
def unpack_geo(geo_dict):
    return geo_dict["type"], "|".join([str(x) for x in geo_dict["bbox"]])
def unpack_geo_2(geo_dict):
    return geo_dict["place_id"]

In [205]:
def query_tweets(query):    
    tweets, media= fetch_all_data(query, 30, '')

    #tweet_df = pd.DataFrame(data=[tweet for sublist in tweets for tweet in sublist])
    #media_df = pd.DataFrame(data=[item for sublist in media for item in sublist])
    

    #tweet_df[["retweet_count", "reply_count", "like_count", "quote_count"]] = tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    #tweet_df["media_keys"] = tweet_df.loc[tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    #del tweet_df["public_metrics"]
    #del tweet_df["attachments"]
        
    #tweet_df.text = tweet_df.text.apply(lambda x: re.sub(r"[^A-Za-z0-9\w\s:@,]", "", x))
    return tweets, media

In [206]:
all_tweets = all_media = pd.DataFrame()
'''for i in ['from:Fridays4future', 'from:FridayForFuture', 'from:GretaThunberg', 'from:Luisamneubauer', 'EndCoal',
    'EndFossilFuels', 
    'PeopleNotProfit', 'NoMoreEmptyPromises', 'UprootTheSystem', 'FridaysForFuture', 
    'ClimateAction', 'ClimateJustice', 'ClimateEmergency', 'ClimateStrike', 'SaveThePlanet', 'climatescam', 
    'climatechangehoax', 'fakeclimate', 'climatehoax', 'globalwarmingisahoax', 'ClimateCrisis', 
    'ClimateChange', 'Climate', 'GlobalWarming']:
    print(i)'''
tweet_df, media_df = query_tweets(i)
all_tweets = pd.concat([all_user, tweet_df])
all_media = pd.concat([all_user, media_df])
    

Rate limit exceeded. Sleeping for 877 seconds.


KeyboardInterrupt: 

In [208]:
tweet_df, media_df = query_tweets('from:Fridays4future')

Rate limit exceeded. Sleeping for 511 seconds.


KeyboardInterrupt: 

In [146]:
responses = tweepy.Paginator(
        client.search_all_tweets,
        query=query,
        max_results=100
    ).flatten()

In [184]:
data = []

for tweet in tweepy.Paginator(client.search_all_tweets, query=query, tweet_fields=['context_annotations', 'created_at'],
                                     max_results=100).flatten(limit=2000):
    


1531627125048066049


AttributeError: 'NoneType' object has no attribute 'append'

In [ ]:
response = tweepy.Paginator(
        client.search_all_tweets,
        query=hashtag,
        start_time=start_time,
        end_time=end_time,
        expansions=expansions,
        tweet_fields=tweet_fields,
        max_results=100
    ).flatten()

In [169]:
len(data)

100

In [148]:
asd = list(responses)

Rate limit exceeded. Sleeping for 748 seconds.


KeyboardInterrupt: 

In [110]:
#specify hashtags to look up
query = 'Fridays4future -is:retweet'

In [185]:
#returns tweets plus metadata for specific tweets, specified as query
def get_tweets_by_hashtag(hashtag, start_time='2022-05-15', end_time: str = None, fmt="%Y-%m-%d"):
    expansions = ["author_id"]
    tweet_fields = ["created_at", "geo", "public_metrics", "text", "context_annotations", "entities"]
    start_time = datetime.datetime.strptime(start_time, fmt)
    if end_time:
        end_time = datetime.datetime.strptime(end_time, fmt)
    else: 
        datetime.datetime.now()
    response = tweepy.Paginator(
        client.search_all_tweets,
        query=hashtag,
        start_time=start_time,
        end_time=end_time,
        expansions=expansions,
        tweet_fields=tweet_fields,
        max_results=100
    ).flatten(limit=10000)
    response = list(response)
    tweets = [
        {
            "id": r.id,
            "author_id": r.author_id,
            "created_at": r.created_at,
            "geo": r.geo,
            "retweet_count": r.public_metrics["retweet_count"],
            "reply_count": r.public_metrics["reply_count"],
            "like_count": r.public_metrics["like_count"],
            "qoute_count": r.public_metrics["quote_count"],
            "text": r.text,
            "entities": str(r.entities),
            "context_annotations": str(r.context_annotations)
        } for r in response
    ]
    tweets = pd.DataFrame(tweets)
    tweets.to_csv("tweets_from_hashtags.csv", index=False, header=False, mode="a")
    return tweets

In [186]:
df_hashtags = get_tweets_by_hashtag(query)
df_hashtags.shape


Rate limit exceeded. Sleeping for 8 seconds.
Rate limit exceeded. Sleeping for 897 seconds.


KeyboardInterrupt: 

In [ ]:
start = datetime.datetime(2022, 5, 1)
end = datetime.datetime(2022, 5, 31)
delta = datetime.timedelta(days=7)


In [97]:
for i in range((end - start).days):
    start_date = end - delta
    end_date = end
    end -= delta
    get_tweets_by_hashtag("ClimateAction", start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
    print(start_date, end_date)
    if i > 5:
        break

Rate limit exceeded. Sleeping for 901 seconds.


KeyboardInterrupt: 

In [ ]:
datetime.datetime.today().date()

In [52]:
pd.read_csv("data/tweets_20220519_1247.csv", on_bad_lines="warn")

,author_id,conversation_id,created_at,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys
0,1068337838,1527219801147461632,2022-05-19 09:29:09+00:00,NaN,1527219801147461632,Dieses Video werden die Aktionär:innen der @De...,50.0,15.0,295.0,2.0,NaN
1,1068337838,1527214056746229773,2022-05-19 09:06:20+00:00,NaN,1527214056746229773,Ein ganz normaler Tag im Klimakollaps: Mit de...,215.0,100.0,1427.0,5.0,NaN
2,1068337838,1527209125851037696,2022-05-19 08:46:44+00:00,NaN,1527209125851037696,"RT @350Deutschland: In Germany, the @KoalaKoll...",36.0,0.0,0.0,0.0,NaN
3,1068337838,1527207260015013889,2022-05-19 08:39:19+00:00,NaN,1527207260015013889,"RT @350Deutschland: 1200 km Ölpipeline, 216000...",31.0,0.0,0.0,0.0,NaN
4,1068337838,1527204602030047232,2022-05-19 08:28:45+00:00,NaN,1527204602030047232,RT @350Deutschland: Die @DeutscheBank untersch...,23.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
30742,1006419421244678144,1007142657720836096,2018-06-14 06:08:16+00:00,NaN,1007142657720836096,RT @daniel_bensson: Obey the law Move the mon...,15.0,0.0,0.0,0.0,NaN
30743,1006419421244678144,1007142570839957504,2018-06-14 06:07:55+00:00,NaN,1007142570839957504,RT @PerArnesjo: Slutsatsen: kött och mjölkprod...,18.0,0.0,0.0,0.0,NaN
30744,1006419421244678144,1006780456539389953,2018-06-13 06:09:00+00:00,NaN,1006780456539389953,"RT @circulareconomy: For example, the fashion ...",70.0,0.0,0.0,0.0,NaN
30745,1006419421244678144,1006584165947576323,2018-06-12 17:09:01+00:00,NaN,1006584165947576323,RT @ParHolmgren: Varmare atmosfär och världsha...,56.0,0.0,0.0,0.0,NaN
